In [23]:
import pandas as pd
import numpy as np

In [37]:
rng = np.random.default_rng()

# Matches Generation

In [24]:
def generate_matches(opponents_array, num_games=10, start_date="2025-01-15", freq="W"):
    dates = pd.date_range(start_date, periods=num_games, freq=freq)
    selected_opponents = np.random.choice(opponents_array, size=num_games)
    
    matches = pd.DataFrame({
        "date": dates,
        "opponent": selected_opponents
    })
    
    return matches

In [25]:
opponents = ["Fábrica de Azulejos Coentrão", "Comunidade Judaica de Vila Real", "Sporting Clube de Cascos de Rolha", "Associação Desportiva de Cascos de Rolha"]
matches_df = generate_matches(opponents, num_games=10, freq="W")
matches_df.insert(0, "match_id", np.arange(len(matches_df), dtype=int))
matches_df

,match_id,date,opponent
0,0,2025-01-19,Fábrica de Azulejos Coentrão
1,1,2025-01-26,Associação Desportiva de Cascos de Rolha
2,2,2025-02-02,Fábrica de Azulejos Coentrão
3,3,2025-02-09,Associação Desportiva de Cascos de Rolha
4,4,2025-02-16,Fábrica de Azulejos Coentrão
5,5,2025-02-23,Fábrica de Azulejos Coentrão
6,6,2025-03-02,Fábrica de Azulejos Coentrão
7,7,2025-03-09,Sporting Clube de Cascos de Rolha
8,8,2025-03-16,Comunidade Judaica de Vila Real
9,9,2025-03-23,Fábrica de Azulejos Coentrão


# Appearances Generation

In [ ]:
def generate_appearances(matches_df, starter_id=0, starter_weight=0.7, substitute_chance=0.15, available_players=[0, 1, 2, 3]):
    appearances = []

    for _, match in matches_df.iterrows():
        match_id = match["match_id"]
        players_in_match = []  # Track players already used in this match

        num_appearances = 2 if np.random.random() < substitute_chance else 1

        for appearance_num in range(num_appearances):
            # Pool of players not yet used in this match
            available_pool = [p for p in available_players if p not in players_in_match]
            if not available_pool:
                break  # No unique players left

            # Prefer starter if available and weighted
            prefer_starter = (np.random.random() < starter_weight) and (starter_id in available_pool)
            if prefer_starter:
                player_id = starter_id
            else:
                # Choose from non-starters first; fall back to any available if necessary
                non_starter_pool = [p for p in available_pool if p != starter_id]
                choice_pool = non_starter_pool if non_starter_pool else available_pool
                player_id = np.random.choice(choice_pool)

            players_in_match.append(player_id)
            appearances.append({
                "match_id": match_id,
                "player_id": player_id,
                "appearance_number": appearance_num + 1
            })

    appearances_df = pd.DataFrame(appearances)
    appearances_df.insert(0, "appearance_id", np.arange(len(appearances_df), dtype=int))
    return appearances_df

In [31]:
appearances_df = generate_appearances(matches_df, starter_id=0, starter_weight=0.7, substitute_chance=0.15)
appearances_df

,appearance_id,match_id,player_id,appearance_number
0,0,0,0,1
1,1,1,1,1
2,2,2,0,1
3,3,2,2,2
4,4,3,0,1
5,5,4,0,1
6,6,5,0,1
7,7,6,0,1
8,8,7,0,1
9,9,8,0,1


In [32]:
dup_check = appearances_df.groupby(["match_id", "player_id"]).size().reset_index(name="count")
dup_check[dup_check["count"] > 1]

,match_id,player_id,count


# Shots Generation

In [35]:
players_df = pd.read_excel("Synthetic Data.xlsx")
players_df.head()

,player_id,name,age,base_torso_x,base_torso_y,upper_arm_length,forearm_length,thigh_length,shin_length,head_vertical_offset,shoulder_horizontal_offset,shoulder_vertical_offset,hip_horizontal_offset,hip_vertical_offset,agility,presence,flexibility,reflexes
0,0,Pedro Farelo,29,13,15,3.5,3.0,6.0,4.5,2.5,2.0,1.5,1.5,3.0,90,85,85,80
1,1,Vasco Palmeirim,23,13,15,3.3,3.1,6.0,4.3,2.4,1.8,1.4,1.5,2.9,80,90,80,75
2,2,José Saramago,38,11,15,3.6,3.0,6.1,4.5,2.5,2.0,1.7,1.5,3.0,70,90,90,60
3,3,Eça de Queirós,31,13,14,3.5,2.9,6.2,4.6,2.6,2.0,1.5,1.5,3.1,75,65,70,70


In [51]:
def generate_shots(appearances_df, players_df, shots_per_appearance_range=(50, 75)):
    shots = []
    start_time=pd.Timestamp("2026-01-01 00:00")
    
    for _, appearance in appearances_df.iterrows():
        appearance_id = appearance["appearance_id"]
        player_id = appearance["player_id"]
        
        player = players_df[players_df["player_id"] == player_id].iloc[0]
        presence = player["presence"]
        
        x_min = 0 + presence * 0.01
        x_max = 27 - presence * 0.01
        y_min = 0 + presence * 0.01
        y_max = 27 - presence * 0.01
        
        num_shots = np.random.randint(shots_per_appearance_range[0], shots_per_appearance_range[1] + 1)
        
        for _ in range(num_shots):
            shots.append({
                "appearance_id": appearance_id,
                "match_id": appearance["match_id"],
                "player_id": player_id,
                "x": rng.uniform(x_min, x_max),
                "y": rng.uniform(y_min, y_max),
                "velocity": rng.lognormal(mean=4.55, sigma=0.16),
                "timestamp": start_time + pd.to_timedelta(rng.uniform(0, 3600),unit="s")
            })
    
    shots_df = pd.DataFrame(shots)
    shots_df.insert(0, "shot_id", np.arange(len(shots_df), dtype=int))
    return shots_df

In [52]:
shots_df = generate_shots(appearances_df, players_df)
shots_df.head(15)

,shot_id,appearance_id,match_id,player_id,x,y,velocity,timestamp
0,0,0,0,0,3.400249,6.048919,118.531590,2026-01-01 00:44:20.648598154
1,1,0,0,0,1.740330,5.936909,96.954680,2026-01-01 00:43:20.004989917
2,2,0,0,0,18.796736,3.589386,93.675243,2026-01-01 00:07:43.858807924
3,3,0,0,0,23.051112,9.037659,98.304861,2026-01-01 00:31:12.806854811
4,4,0,0,0,18.044147,6.989478,135.088925,2026-01-01 00:57:07.735080391
5,5,0,0,0,25.890814,11.226589,87.032263,2026-01-01 00:30:49.603603779
6,6,0,0,0,11.213912,8.191412,118.915623,2026-01-01 00:06:12.892325370
7,7,0,0,0,9.646961,12.407752,102.859036,2026-01-01 00:18:49.884901021
8,8,0,0,0,13.668042,14.903526,94.220810,2026-01-01 00:30:50.406186589
9,9,0,0,0,17.087568,21.919203,86.589178,2026-01-01 00:51:48.912719602
